In [8]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import spacy
import wikipedia

import pandas as pd
import glob
import os
from tqdm import tqdm

from collections import Counter

import itertools

import string


In [9]:
allowed = set(string.ascii_lowercase + string.ascii_uppercase+ string.digits + "'")

import re

s="elon musk"

print(bool(re.match('^[a-zA-Z0-9]+$', s)))

False


In [10]:
stop_words = set(stopwords.words('english'))

In [11]:
nlp = spacy.load('en_core_web_lg')


In [12]:
df=pd.read_excel(r"H:\NM\05_22\edu11112022\arxiv_education.xlsx",index_col=0)
df

,prior,title,text,article_link
0,Education%20System,"21st Century Ergonomic Education, From Little ...","Despite intense efforts, contemporary educatio...",http://arxiv.org/abs/1403.0281v1
1,Education%20System,Methodical Aspects of Informatization of Physi...,The article considers the methodical aspects o...,http://arxiv.org/abs/1504.02837v1
2,Education%20System,Interconnectedness in Education Systems,"Underlying complex systems, there is a rich we...",http://arxiv.org/abs/2203.05624v2
3,Education%20System,How to Teach a Teacher: Challenges and Opportu...,Preparing future physics teachers for the dema...,http://arxiv.org/abs/2201.08682v1
4,Education%20System,he International Monetary Funds intervention i...,Enabling children to acquire an education is o...,http://arxiv.org/abs/2201.00013v1
...,...,...,...,...
5,Situated%20Learning%20Theory%20,Statistical Mechanics of Dictionary Learning,Finding a basis matrix (dictionary) by which o...,http://arxiv.org/abs/1203.6178v3
6,Situated%20Learning%20Theory%20,owards Explaining Subjective Ground of Individ...,Large-scale language models have been reducing...,http://arxiv.org/abs/2211.09953v1
7,Situated%20Learning%20Theory%20,Stable variation in multidimensional competition,The Fundamental Theorem of Language Change (Ya...,http://arxiv.org/abs/2003.06265v1
8,Situated%20Learning%20Theory%20,hresholding in Learning Theory,In this paper we investigate the problem of le...,http://arxiv.org/abs/math/0510271v1


In [13]:
BAD_DEP=["poss","det"]
def filter_sentence(sent):
    doc = nlp(sent)
    filtered_sentence=[]
    for noun_chunk in doc.noun_chunks:
        original_chunk=noun_chunk
        if noun_chunk[0].dep_ in BAD_DEP:
            noun_chunk=noun_chunk[1:]
        
        w=noun_chunk.text.lower()
        ent=noun_chunk.root.ent_type_
        w=noun_chunk.text.lower()
#         if w not in stop_words:
        # if check(w):
        #     print(w)
        filtered_sentence.append({"word":w,
                                      "entity":noun_chunk.root.ent_type_,
                                      "dependency":noun_chunk.root.dep_,
                                     "original_word":original_chunk.text})
            

    return filtered_sentence
filter_sentence("'s Elon Musk owes me 5000 dollars")

[{'word': 'elon musk',
  'entity': 'PERSON',
  'dependency': 'nsubj',
  'original_word': 'Elon Musk'},
 {'word': 'me', 'entity': '', 'dependency': 'dative', 'original_word': 'me'},
 {'word': '5000 dollars',
  'entity': 'MONEY',
  'dependency': 'dobj',
  'original_word': '5000 dollars'}]

In [16]:
words_abs=[]
words_title=[]
abs_cluster_list=[-1,0,1,2,8]
for row in df.iterrows():
    # if row[1]["abstract_topics"] in abs_cluster_list:
        abstract=row[1]["text"]
        title=row[1]["title"]
        words_abs=words_abs+filter_sentence(abstract)
        words_title=words_title+filter_sentence(title)

In [17]:
df_abs=pd.DataFrame(words_abs)
df_abs["count"] = 1
df_abs = df_abs.groupby(["word", "entity", "dependency","original_word"])["count"].count().reset_index()
df_abs["len"]=df_abs["word"].str.count(' ') + 1
df_abs.to_excel("abs_word_arxiv.xlsx")

In [18]:
df_abs["entity"].value_counts()

               24518
ORG             1104
PERSON           339
GPE              164
DATE              97
PRODUCT           74
WORK_OF_ART       68
PERCENT           47
NORP              46
CARDINAL          30
LOC               23
EVENT             21
FAC               20
MONEY             18
LAW               16
TIME               4
LANGUAGE           3
QUANTITY           3
Name: entity, dtype: int64